In [1]:
import numpy as np
import pandas as pd
import cv2
import random
import csv
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

In [2]:
base_path = '../../datasets/german_traffic_signs/'
train_path = base_path + 'imgs/train'
test_path = base_path + 'imgs/test'

In [18]:
imgpaths = [os.path.join(root, f).replace('\\', '/')
            for root, _, files in os.walk(train_path) 
            for f in files if f.endswith('.ppm')]
random.shuffle(imgpaths)

In [21]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(imgpaths, test_size=0.1, random_state=42)

n_train, n_valid = len(train), len(valid)
n_classes = len(list(open(base_path + 'signnames.csv'))) - 1

print("Number of training examples =", n_train)
print("Number of validation examples =", n_valid)
print("Number of classes =", n_classes)

Number of training examples = 35288
Number of validation examples = 3921
Number of classes = 43
